In [ ]:
import requests

In [10]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('reported_flows.db')
df = pd.read_sql_query(
    "SELECT * FROM reported_flows ORDER BY reported_at DESC LIMIT 100",
    conn
)
conn.close()


In [11]:
df

,id,flow_key,src_ip,dst_ip,sport,dport,protocol,total_bytes,pkt_count,prediction,label_humain,user_id,reported_at,flow_data


In [ ]:
print(df.flow_data.values[0])

In [ ]:
# Flow: 172.30.0.7:36666 → 172.30.0.5:80 (TCP)
# Flow: 172.30.0.7:36682 → 172.30.0.5:80 (TCP)
# Flow: 172.30.0.7:36692 → 172.30.0.5:80 (TCP)
# Flow: 172.30.0.7:51110 → 172.30.0.5:80 (TCP)
# Flow: 172.30.0.7:51116 → 172.30.0.5:80 (TCP)
# Flow: 172.30.0.7:51130 → 172.30.0.5:80 (TCP)


In [1]:
import sqlite3

def recreate_reported_flows_table():
    try:
        conn = sqlite3.connect('reported_flows.db')
        cursor = conn.cursor()

        # Supprimer l'ancienne table
        cursor.execute('DROP TABLE IF EXISTS reported_flows')

        # Créer la nouvelle table
        cursor.execute('''
        CREATE TABLE reported_flows (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            flow_key TEXT,
            src_ip TEXT,
            dst_ip TEXT,
            sport INTEGER,
            dport INTEGER,
            protocol TEXT,
            total_bytes INTEGER,
            pkt_count INTEGER,
            prediction TEXT,
            label_humain TEXT,
            user_id INTEGER,
            reported_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            flow_data TEXT
        )
        ''')

        conn.commit()
        conn.close()
        print("La table reported_flows a été recréée avec succès")

    except Exception as e:
        print(f"Erreur lors de la recréation de la table : {e}")

# Exécuter la fonction
if __name__ == "__main__":
    recreate_reported_flows_table()

La table reported_flows a été recréée avec succès


In [12]:
import json
import pandas as pd
import sqlite3


conn = sqlite3.connect('reported_flows.db')

df = pd.read_sql_query(
    "SELECT * FROM reported_flows ORDER BY reported_at DESC LIMIT 100",
    conn
)
feature_cols = [
    "total_bytes",
    "pkt_count",
    "psh_count",
    "fwd_bytes",
    "bwd_bytes",
    "fwd_pkts",
    "bwd_pkts",
    "dport",
    "duration_ms",
    "flow_pkts_per_s",
    "fwd_bwd_ratio"
]

extracted_rows = []

# Parcourir chaque ligne du DataFrame
for idx, row in df.iterrows():
    # Charger le JSON contenu dans flow_data
    flow_dict = json.loads(row["flow_data"])

    # Extraire uniquement les champs souhaités
    row_features = {
        col: flow_dict.get(col, 0)
        for col in feature_cols
    }

    # Inclure aussi label_humain (qui se trouve dans le DataFrame)
    row_features["label_humain"] = row["label_humain"]

    extracted_rows.append(row_features)

# Créer un nouveau DataFrame avec les features + label_humain
df_features = pd.DataFrame(extracted_rows)

# Vérifier les premières lignes


In [13]:
df_features

,total_bytes,pkt_count,psh_count,fwd_bytes,bwd_bytes,fwd_pkts,bwd_pkts,dport,duration_ms,flow_pkts_per_s,fwd_bwd_ratio,label_humain
0,920,12,3,67,853,7,5,80,731.973410,16.394038,0.078546,Normal
1,920,12,3,67,853,7,5,80,87.926865,136.477060,0.078546,Normal
2,920,12,3,67,853,7,5,80,107.424259,111.706612,0.078546,Normal
3,920,12,3,67,853,7,5,80,96.950531,123.774464,0.078546,Normal
4,920,12,3,67,853,7,5,80,95.770836,125.299105,0.078546,Normal
5,920,12,3,67,853,7,5,80,105.536699,113.704523,0.078546,Normal


In [4]:
df_features.columns

Index(['total_bytes', 'pkt_count', 'psh_count', 'fwd_bytes', 'bwd_bytes',
       'fwd_pkts', 'bwd_pkts', 'dport', 'duration_ms', 'flow_pkts_per_s',
       'fwd_bwd_ratio', 'label_humain'],
      dtype='object')